실습 과제 : Electro 사이트를 크롤링해서 category, name,   link, price 4개 컬럼으로 구성되는 데이터프레임을 출력하세요.

In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re

url = 'https://startcoding.pythonanywhere.com/basic?page={}&keyword='

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 컬럼 생성
category = []
name = []
link = []
price = []
for i in range(1,5):
    response = requests.get(url.format(i), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cate = soup.select('p.product-category')
    na = soup.select('h3.product-name')
    li = soup.select('h3.product-name a')
    pri = soup.select('h4.product-price')
    for c in cate:
        category.append(c.text)
    for n in na:
        name.append(n.text)
    for l in li:
        link.append(l['href'])
    for p in pri:
        if p.find_all('del'):
            for del_tag in p.find_all('del'):
                del_tag.decompose() 
        p = p.text.strip().replace(',','')
        p = re.findall('\d',p) # replace('원','')로도 가능
        price.append(''.join(p))

# 데이터프레임 생성
df = pd.DataFrame({'category': category,
                   'name':name,
                   'link':link,
                   'price':price})
df.head()

,category,name,link,price
0,노트북,"에이서 스위프트 GO 16 OLED, 스틸 그레이, 코어i7, 512GB, 16GB...",#product1_detail.html,1419000
1,노트북,"삼성전자 노트북 플러스2 15.6, 퓨어 화이트, NT550XDA-K24AT, 펜티...",#product2_detail.html,549000
2,노트북,"레노버 아이디어패드 슬림 1 15AMN7 15.6, 256GB, Free DOS, ...",#product3_detail.html,529000
3,노트북,"레노버 V15 G4 AMN 15.6, Arctic Grey, 라이젠3, 256GB,...",#product4_detail.html,624000
4,노트북,"LG 울트라PC 엣지 16, 차콜 그레이, 라이젠5, 256GB, 16GB, WIN...",#product5_detail.html,1135000


In [ ]:
df.category.value_counts()

category
노트북     10
데스크탑    10
휴대폰     10
모니터      5
Name: count, dtype: int64

In [ ]:
# price가 두개 일 수 있으므로 최신 가격 선택
soup.select_one('.product-price').text.split('원')[0].replace(',','')

'89000'

In [ ]:
import requests
from bs4 import BeautifulSoup 
import re


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data = []
for i in range(1,5):
    response = requests.get(f'https://startcoding.pythonanywhere.com/basic?page={i}&keyword=', headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('.product')


    for item in items:
        category = item.select_one('.product-category').text.strip()
        name = item.select_one('.product-name').text.strip()
        link = item.select_one('.product-name > a').attrs['href']
        price = item.select_one('.product-price').text.split('원')[0].replace(',','')
        data.append([category, name, link, price])

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns = ['category','name','link','price'])
df.head()

,category,name,link,price
0,노트북,"에이서 스위프트 GO 16 OLED, 스틸 그레이, 코어i7, 512GB, 16GB...",#product1_detail.html,1419000
1,노트북,"삼성전자 노트북 플러스2 15.6, 퓨어 화이트, NT550XDA-K24AT, 펜티...",#product2_detail.html,549000
2,노트북,"레노버 아이디어패드 슬림 1 15AMN7 15.6, 256GB, Free DOS, ...",#product3_detail.html,529000
3,노트북,"레노버 V15 G4 AMN 15.6, Arctic Grey, 라이젠3, 256GB,...",#product4_detail.html,624000
4,노트북,"LG 울트라PC 엣지 16, 차콜 그레이, 라이젠5, 256GB, 16GB, WIN...",#product5_detail.html,1135000


In [ ]:
# UTF-8 인코딩으로 데이터프레임을 CSV 파일로 저장하는데, 이때 UTF-8 BOM을 포함하여 저장
df.to_csv('result.csv',encoding='utf-8-sig',index=False)